In [ ]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
titanic = pd.read_csv('/kaggle/input/titanic/train.csv')

In [ ]:
titanic.head()

In [ ]:
numeric_features = ["Pclass","Age","SibSp","Parch","Fare"]
categoric_features = ["Sex","Ticket","Embarked"]
features = ["Pclass","Age","SibSp","Parch","Fare","Sex","Ticket","Embarked"]
label = "Survived"

In [ ]:
titanic = titanic.drop(columns=["Name","Cabin"])

In [ ]:
titanic.isnull().sum()

In [ ]:
titanic["Age"].fillna(titanic["Age"].mean(), inplace = True) 

In [ ]:
titanic = titanic.dropna()

In [ ]:
numerical = [0,1,2,3,4]
categorical = [5,6,7]

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

for col in numeric_features:
    titanic.boxplot(column=col, by=label, figsize=(6,6))
    plt.title(col)
plt.show()

In [ ]:
for col in categoric_features:
    counts = titanic[col].value_counts().sort_index()
    fig = plt.figure(figsize=(9, 6))
    ax = fig.gca()
    counts.plot.bar(ax = ax, color='steelblue')
    ax.set_title(col + ' counts')
    ax.set_xlabel(col) 
    ax.set_ylabel("Frequency")
plt.show()

In [ ]:
for col in categoric_features:
    fig = plt.figure(figsize=(9, 6))
    ax = fig.gca()
    titanic.boxplot(column = label, by = col, ax = ax)
    ax.set_title('Label by ' + col)
    ax.set_ylabel("Survived")
plt.show()

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np

In [ ]:
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

In [ ]:
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical),
        ('cat', categorical_transformer, categorical)])


In [ ]:
reg = 0.01
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('logregressor', LogisticRegression(C=1/reg, solver="liblinear"))])

In [ ]:
new_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('logregressor', GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0))])

In [ ]:
X, y = titanic[numeric_features+categoric_features].values, titanic[label].values

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

print ('Training cases: %d\nTest cases: %d' % (X_train.shape[0], X_test.shape[0]))

In [ ]:
model = pipeline.fit(X_train, (y_train))
print (model)

In [ ]:
new_model = new_pipeline.fit(X_train,(y_train))

In [ ]:
predictions = model.predict(X_test)

In [ ]:
new_predictions = new_model.predict(X_test)

In [ ]:
print('Predicted labels: ', predictions)
print('Actual labels:    ' ,y_test)

In [ ]:
print('Predicted labels: ', new_predictions)
print('Actual labels:    ' ,y_test)

In [ ]:
from sklearn.metrics import accuracy_score

print('Accuracy: ', accuracy_score(y_test, predictions))

In [ ]:
print('Accuracy: ', accuracy_score(y_test, new_predictions))

In [ ]:
from sklearn. metrics import classification_report

print(classification_report(y_test, predictions))

In [ ]:

from sklearn.metrics import precision_score, recall_score

print("Overall Precision:",precision_score(y_test, predictions))
print("Overall Recall:",recall_score(y_test, predictions))

In [ ]:
import joblib

# Save the model as a pickle file
filename = '/kaggle/working/titanic.pkl'
joblib.dump(model, filename)

In [ ]:
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')

In [ ]:
test_data.head()

In [ ]:
test_data = test_data.drop(columns=["Name","Cabin"])

In [ ]:
test_data.isnull().sum()

In [ ]:
test_data["Age"].fillna(test_data["Age"].mean(), inplace = True) 

In [ ]:
test_data["Fare"].fillna(test_data["Fare"].mean(),inplace= True)

In [ ]:
X_train

In [ ]:
test_X = test_data[numeric_features+categoric_features].values

In [ ]:
predict_val = model.predict(test_X)

In [ ]:
result = []
for i,k in test_data.iterrows():
    result.append([k["PassengerId"],predict_val[i]])

In [ ]:
df = pd.DataFrame(result,columns=["PassengerId","Survived"])

In [ ]:
df.to_csv('/kaggle/working/output.csv',index=False)